<a href="https://colab.research.google.com/github/sanjanasrinivas22/1BM23CS301-AI/blob/main/FOL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
print("Sanjana Srinivas-1BM23CS301")
def unify(x, y, substitutions=None):
    if substitutions is None:
        substitutions = {}

    if x == y:
        return substitutions

    # If x is a variable
    if is_variable(x):
        return unify_var(x, y, substitutions)

    # If y is a variable
    if is_variable(y):
        return unify_var(y, x, substitutions)

    # If both are compound terms (like f(a), g(X), etc.)
    if isinstance(x, tuple) and isinstance(y, tuple):
        if x[0] != y[0] or len(x[1]) != len(y[1]):
            return None
        for xi, yi in zip(x[1], y[1]):
            substitutions = unify(apply(substitutions, xi), apply(substitutions, yi), substitutions)
            if substitutions is None:
                return None
        return substitutions
    return None


def unify_var(var, x, substitutions):
    if var in substitutions:
        return unify(substitutions[var], x, substitutions)
    elif occurs_check(var, x, substitutions):
        return None
    else:
        substitutions[var] = x
        return substitutions


def occurs_check(var, x, substitutions):
    if var == x:
        return True
    elif isinstance(x, tuple):
        return any(occurs_check(var, xi, substitutions) for xi in x[1])
    elif isinstance(x, str) and x in substitutions:
        return occurs_check(var, substitutions[x], substitutions)
    return False


def apply(substitutions, expr):
    if isinstance(expr, str):
        return substitutions.get(expr, expr)
    elif isinstance(expr, tuple):
        return (expr[0], [apply(substitutions, e) for e in expr[1]])
    else:
        return expr


def is_variable(x):
    # A variable is a single lowercase letter (e.g., x, y, z)
    return isinstance(x, str) and len(x) == 1 and x.islower()


def parse(expr):
    expr = expr.replace(" ", "")
    if '(' not in expr:
        return expr
    functor = expr.split('(')[0]
    args = expr[len(functor) + 1:-1]
    parts, depth, start = [], 0, 0
    for i, c in enumerate(args):
        if c == ',' and depth == 0:
            parts.append(args[start:i])
            start = i + 1
        elif c == '(':
            depth += 1
        elif c == ')':
            depth -= 1
    parts.append(args[start:])
    return (functor, [parse(p) for p in parts])


# -------- TEST CASES --------
tests = [
    ("p(b,x,f(g(z)))", "p(z,f(y),f(y))"),  # fixed the extra parenthesis
    ("Q(a,g(x,a),f(y))", "Q(a,g(f(b),a),x)"),
    ("p(f(a),g(Y))", "p(X,X)"),
    ("prime(11)", "prime(y)"),
    ("knows(John,x)", "knows(y,mother(y))"),
    ("knows(John,x)", "knows(y,Bill)")
]

print(" UNIFICATION RESULTS \n")
for i, (a, b) in enumerate(tests, start=1):
    e1 = parse(a)
    e2 = parse(b)
    result = unify(e1, e2)
    print(f"Q{i}: {a}  AND  {b}")
    if result is None:
        print("MGU = FAIL\n")
    else:
        print("MGU =", result, "\n")


Sanjana Srinivas-1BM23CS301
 UNIFICATION RESULTS 

Q1: p(b,x,f(g(z)))  AND  p(z,f(y),f(y))
MGU = {'b': 'z', 'x': ('f', ['y']), 'y': ('g', ['z'])} 

Q2: Q(a,g(x,a),f(y))  AND  Q(a,g(f(b),a),x)
MGU = {'x': ('f', ['b']), 'y': 'b'} 

Q3: p(f(a),g(Y))  AND  p(X,X)
MGU = FAIL

Q4: prime(11)  AND  prime(y)
MGU = {'y': '11'} 

Q5: knows(John,x)  AND  knows(y,mother(y))
MGU = {'y': 'John', 'x': ('mother', ['John'])} 

Q6: knows(John,x)  AND  knows(y,Bill)
MGU = {'y': 'John', 'x': 'Bill'} 

